In [ ]:
import requests
from bs4 import BeautifulSoup
from twilio.rest import Client
import configparser

In [ ]:
dashboardUrl = None
loginUrl = None
loginValues = None
phoneNumbersList = None
twilioNumber = None

In [ ]:
def parseConfigFile():
    config = configparser.ConfigParser()
    config.read('config.ini')
    #config.sections()
    global dashboardUrl 
    dashboardUrl = config['urls']['dashboardUrl']
    
    global loginUrl
    loginUrl = config['urls']['loginUrl']
    
    global loginValues
    loginValues = {'property': config['flatKeys']['property'],
    'flat': config['flatKeys']['flat'] }
    
    global phoneNumbersList
    phoneNumbersList = config['personalInfo']['phoneNumbers'].split(",")
    
    global twilioNumber
    twilioNumber = config['personalInfo']['twilioNumber']

In [ ]:
def createSession():
    session = requests.session()
    session.post(loginUrl, data=loginValues)
    return session

In [ ]:
def fetchDashboard(session):
    dashboard = session.get(dashboardUrl)
    return dashboard

In [ ]:
def parseDashboard(dashboard):
    dashboardHtml = dashboard.content
    parsedDashboard = BeautifulSoup(dashboard.content,"html.parser")
    return parsedDashboard

In [ ]:
def getTotalBill(parsedDashboard):
    totalBill = ""
    for tag in parsedDashboard.find_all('h3', recursive=True):
        #print(tag.contents[2])
        totalBill = totalBill +  str(tag.contents[2])[2:]
    return totalBill

In [ ]:
def getYesterdaysBill(parsedDashboard):
    htmlContent = ""
    for tag in parsedDashboard.find_all('li', recursive=True):
        if("Grid" in str(tag.contents)):
            #print(h.contents)
            htmlContent = htmlContent + (str(tag.contents[1])) 
            break
    #print(stri)
    bill = []
    bill.append(htmlContent.split("<strong>")[1].split("</strong>")[0] + htmlContent.split("<br/>")[1].split("</div>")[0])
    bill.append(htmlContent.split("KWH(G) : ")[1].split("</span>")[0])
    bill.append(htmlContent.split("</i>: ")[1].split("</span>")[0])
    
    return bill

In [ ]:
def sendMessage(totalBill, yesterdaysBill):
    # Your Account Sid and Auth Token from twilio.com/console
    account_sid = 'ACda6d35f15973666d4e157fb4ee9463bf'
    auth_token = '048740d1d43d7db9aa9b77f0d43c108a'
    client = Client(account_sid, auth_token)
    
    messageBody = "Your total bill is Rs. " + totalBill + ". Your bill for " + yesterdaysBill[0] + "is Rs. " + yesterdaysBill[2] + ". Units spent until "  + yesterdaysBill[0] + "are " + yesterdaysBill[1] + "KWH."
    print(messageBody)
    
    for phoneNumber in phoneNumbersList:
        message = client.messages.create(
                                  body=messageBody,
                                  from_=twilioNumber,
                                  to=phoneNumber
                                  )
        print("Message sent to " + phoneNumber)

    #print(message.sid)

In [ ]:
def main():
    parseConfigFile()
    session = createSession()
    dashboard = fetchDashboard(session)
    parsedDashboard = parseDashboard(dashboard)
    totalBill = getTotalBill(parsedDashboard)
    yesterdaysBill = getYesterdaysBill(parsedDashboard)
    sendMessage(totalBill,yesterdaysBill)

In [ ]:
if __name__ == "__main__":main()